In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

In [ ]:
%%cu
#include <stdio.h>
#include <iostream>
#include <math.h>
#include <chrono> 
#include <cuda.h>

using namespace std;

__device__ double func(double x){
  return(x*x + 5.2)/(sin(0.5+0.1*x*x));
}

__global__ void obliczProstokat(double* a, double c, double delta, int N){   
  int idx = blockIdx.x * blockDim.x + threadIdx.x;  
  double x = c + (double)idx * delta;
  if(idx < N){
    a[idx] = func(x) + func(x + delta);
  }
}

__host__ double metodaProstokatow(double a, double b, int N){    
  double delta = (b - a)/ N;
  cudaError_t errorcode = cudaSuccess;
  int size = N * sizeof(double);
  double* tabHost = (double*) malloc(size);
  double* tabDevice;
  if((errorcode = cudaMalloc ((void**)&tabDevice, size)) != cudaSuccess){
    cout << "cudaMalloc(): " << cudaGetErrorString(errorcode) << endl;
    exit(1);
  }
  int blockSize = 256;
  int blockNum = N / blockSize + (N % blockSize == 0 ? 0 : 1);
  obliczProstokat <<< blockNum, blockSize >>> (tabDevice, a, delta, N); 
  if((errorcode = cudaMemcpy(tabHost, tabDevice, sizeof(double) * N, cudaMemcpyDeviceToHost)) != cudaSuccess){
    cout << "cudaMemcpy(): " << cudaGetErrorString(errorcode) << endl;
    exit(1);
  }
  double wynik = 0.0;
  for(int i = 0; i < N; i++){
    wynik += tabHost[i];
  }
  wynik *= delta / 2.0;
  free(tabHost);
  cudaFree(tabDevice);
  return wynik;
}

__global__ void obliczTrapez(double* a, double c, double delta, int N){    
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  double x = c + (double)idx * delta;
  if(idx < N){
    a[idx] = delta + func(x + delta);
  }
}

__host__ double metodaTrapezow(double c, double d, int N){    
  double delta = (d - c)/ N;
  cudaError_t errorcode = cudaSuccess;
  int size = N * sizeof(double);
  double* tabHost = (double*)malloc(size);
  double* tabDevice;
  if((errorcode = cudaMalloc ((void**)&tabDevice, size)) != cudaSuccess){
    cout << "cudaMalloc(): " << cudaGetErrorString(errorcode) << endl;
    exit(1);
  }
  int blockSize = 256;
  int blockNum = N / blockSize + (N % blockSize == 0 ? 0 : 1);
  obliczTrapez <<< blockNum, blockSize >>> (tabDevice, c, delta, N); 
  if((errorcode = cudaMemcpy(tabHost, tabDevice, sizeof(double) * N, cudaMemcpyDeviceToHost)) != cudaSuccess){
    cout << "cudaMemcpy(): " << cudaGetErrorString(errorcode) << endl;
    exit(1);
  }
  double wynik = 0.0;
  for(int i = 0; i < N; i++){
    wynik += tabHost[i];
  }
  wynik *= delta;
  free(tabHost);
  cudaFree(tabDevice);
  return wynik;
}

__device__ void ustawRdz(int* n_start, int* n_end, int n, int blockNum, int blockWidth){   
  int threadId = blockWidth * blockIdx.x + threadIdx.x;
  int nextThreadId = threadId + 1;
  int threads = blockWidth * blockNum;
  *n_start = (threadId * n)/threads;
  *n_end = (nextThreadId * n)/threads;
}

__device__ void obliczParab(int blockNum, int blockWidth, double a, double b, int n, double* result){
  *result = 0;
  double h = (b - a)/n;
  int idx_start;
  int idx_end;
  ustawRdz(&idx_start, &idx_end, n-1, blockNum, blockWidth);
  for(int i = idx_start; i < idx_end; i += 2){
    *result += (func(a  + h * (i - 1)) + 4 * func(a + h * (i)) + func(a + h * (i + 1))) * h / 3;
  }
}

__global__ void obliczSimpson(int blockNum, int blockWidth, double* result, double a, double b, int n){
    
  double wynik = 0;
  obliczParab(blockNum, blockWidth, a, b, n, &wynik);
  result[(blockIdx.x * blockWidth + threadIdx.x)] = wynik;
}

__host__ double metodaSimpsona(double a, double b, int n){
    
  const int blockNum = 32;
  const int blockWidth = 32;
  double tabHost[blockNum*blockWidth] = {0};
  double* tabDevice;
  cudaMalloc ((void**)&tabDevice, sizeof(double) * blockNum * blockWidth);
  obliczSimpson <<< blockNum, blockWidth >>> (blockNum, blockWidth, tabDevice, a, b, n );
  cudaThreadSynchronize();
  cudaMemcpy(tabHost, tabDevice, sizeof(double) * blockNum * blockWidth, cudaMemcpyDeviceToHost);
  double wynik = 0;

  for(int i = 0; i != blockNum *  blockWidth; i++){
    wynik += tabHost[i];
  }
  cudaFree (tabDevice);
  return wynik;
}

int main(){   
  double a = 1;
  double b = 4;
  int n = 10000; 
  auto begin = std::chrono::high_resolution_clock::now();
  double Prostokat = metodaProstokatow(a, b, n);
  double Trapez = metodaTrapezow(a, b, n);
  double Simpson = metodaSimpsona(a, b, n);
  cout << "Wynik Metoda Prostokat: " << Prostokat << endl;
  cout << "Wynik Metoda Trapez: " << Trapez << endl;
  cout << "Wynik Metoda Simpson: " << Simpson << endl;
  auto end = std::chrono::high_resolution_clock::now();
  auto elapsed = std::chrono::duration_cast<std::chrono::nanoseconds>(end - begin);
  printf("Czas wykonania: %.3f seconds.\n", elapsed.count() * 1e-9);
  return 0;
}

In [ ]:
%%cu
#include <stdio.h>
#include <iostream>
#include <math.h>
#include <chrono> 

using namespace std;
double func(double x){
  return (x*x + 5.2)/(sin(0.5+0.1*x*x));
}

double metodaProstokatow(double a, double b, int N){  
  double delta = (b - a) / N;
  double suma = 0, wynik;
	for (int i = 0; i < N; i++){
	  suma += delta * func(a + delta*(i + 0.5));
	}
	wynik = suma;
  return wynik;
}

double metodaTrapezow(double a, double b, int n){  
  double wynik = 0;
  double delta = (b - a) / n;
  int i;
  for(i = 1; i <= n-1; i++){
    wynik += func(a + i * delta);
  }
  wynik += (func(a) + func(b)) / 2;
  wynik *= delta;
  return wynik;
}
double metodaSimpsona(double a, double b, int n){   
  double delta = (b - a) / n, s, wynik, x;
  wynik = 0;
  s = 0;
  for (int i = 1; i < n; i++){
    x = a + i * delta;
    s += func(x - delta / 2);
    wynik += func(x);
  }
  s += func(b - delta / 2);
  wynik = (delta / 6) * (func(a) + func(b) + 2 * wynik + 4 * s);

  return wynik;
}

int main(){
  double a = 1;
  double b = 4;
  int n = 10000;  
  auto begin = std::chrono::high_resolution_clock::now();
  double Prostokat = metodaProstokatow(a, b, n);
  double Trapez = metodaTrapezow(a, b, n); 
  double Simpson = metodaSimpsona(a, b, n);
  cout << "Wynik Metoda Prostokat: " << Prostokat << endl;
  cout << "Wynik Metoda Trapez: " << Trapez << endl;
  cout << "Wynik Metoda Simspon: " << Simpson << endl;
  //stop czas
  auto end = std::chrono::high_resolution_clock::now();
  auto elapsed = std::chrono::duration_cast<std::chrono::nanoseconds>(end - begin);
  printf("Czas wykonania: %.3f seconds.\n", elapsed.count() * 1e-9);

  return 0;
}